# Mapping labels to IRS normalized proteomics results

### Import libraries

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats

import sys
import os

### Import normalized data

In [2]:
ori_df = pd.read_csv('../Intermediate data/IRS_norm_data_no_labels.csv')
ori_df = ori_df.rename(columns={'Unnamed: 0':'fasta_header'})
ori_df.head(50)

,fasta_header,BL11,BL12,BL21,BL22,CF11,CF12,CF21,CF22,POOL1,...,POOL3,POOL4,S111,S112,S121,S122,S711,S712,S721,S722
0,CONT_011|gi|136425|sp|P00760|TRYP_BOVIN (+1),3.835027e+06,3.911514e+06,3.874019e+06,3.901737e+06,3.731196e+06,4.510384e+06,4.588257e+06,4.236523e+06,3.903935e+06,...,4.161023e+06,3.962337e+06,3.784376e+06,3.831416e+06,4.690894e+06,4.209486e+06,3.869808e+06,3.996445e+06,4.224319e+06,5.063390e+06
1,CONT_068|gi|1082558|pir||S41161 (+1),6.199579e+06,7.066370e+06,6.312817e+06,6.931707e+06,5.935731e+06,6.074942e+06,6.754498e+06,1.218101e+07,6.828324e+06,...,6.827600e+06,7.112698e+06,4.896269e+06,7.263273e+06,6.779417e+06,6.442307e+06,4.001432e+06,8.255272e+06,8.388302e+06,8.690283e+06
2,CONT_072|gi|547754|sp|P35908|K22E_HUMAN (+1),1.580745e+06,1.583974e+06,1.860665e+06,2.163203e+06,1.613268e+06,1.448619e+06,1.885271e+06,1.723223e+06,1.812906e+06,...,1.868784e+06,1.834232e+06,8.752198e+05,2.208721e+06,1.820112e+06,1.493082e+06,1.374696e+06,2.530308e+06,2.459852e+06,2.744858e+06
3,CONT_089|gi|71536|pir||KRHU2,1.920025e+06,2.443472e+06,2.154790e+06,2.331452e+06,2.301822e+06,2.177362e+06,2.417178e+06,4.277902e+06,2.414568e+06,...,2.422728e+06,2.404177e+06,1.593816e+06,2.433836e+06,2.269750e+06,2.144563e+06,1.557320e+06,2.699825e+06,3.269226e+06,3.409168e+06
4,CONT_092|gi|71528|pir||KRHU0 (+2),3.971354e+06,4.113378e+06,4.550997e+06,5.087601e+06,4.503819e+06,4.189011e+06,4.710197e+06,5.307276e+06,4.811108e+06,...,4.856246e+06,4.939118e+06,2.985351e+06,5.574157e+06,5.115362e+06,4.021953e+06,4.122463e+06,6.309219e+06,6.940189e+06,6.734396e+06
5,EXTRA_0117 (+1),1.440281e+07,1.976781e+07,1.777404e+07,1.323546e+07,1.061170e+07,1.441676e+07,1.598371e+07,1.290702e+07,3.680170e+07,...,3.594734e+07,3.689755e+07,1.271712e+07,1.192183e+07,1.637376e+07,1.248003e+07,1.228075e+08,1.244547e+08,1.007148e+08,1.100057e+08
6,EXTRA_0118,4.354863e+08,4.575808e+08,4.548598e+08,4.165037e+08,3.489123e+08,4.174036e+08,3.975855e+08,3.707646e+08,8.356862e+08,...,8.523489e+08,8.453678e+08,1.743954e+09,1.499184e+09,1.547220e+09,1.508127e+09,1.096038e+09,1.057116e+09,1.002265e+09,1.096692e+09
7,EXTRA_0119,1.539847e+08,2.001240e+08,1.980018e+08,1.596445e+08,1.080382e+08,2.495085e+08,1.974281e+08,1.836786e+08,8.359638e+08,...,8.565779e+08,8.412198e+08,2.061338e+09,1.934649e+09,1.918383e+09,1.835831e+09,1.232947e+09,1.280982e+09,1.098689e+09,1.286727e+09
8,EXTRA_0121,1.500901e+07,1.762070e+07,1.691547e+07,1.469638e+07,1.207299e+07,1.455292e+07,1.614969e+07,1.252022e+07,3.134000e+07,...,3.051610e+07,3.136489e+07,1.166347e+07,1.177902e+07,1.510625e+07,1.237009e+07,8.914262e+07,9.935665e+07,8.252663e+07,8.880254e+07
9,EXTRA_0125,3.332186e+07,3.613396e+07,3.704996e+07,3.125404e+07,5.372009e+07,5.407240e+07,5.730786e+07,5.168018e+07,6.714330e+07,...,6.490930e+07,6.536691e+07,3.139849e+07,3.339590e+07,3.510298e+07,3.100359e+07,1.433136e+08,1.434786e+08,1.489644e+08,1.757454e+08


### Extract accession numbers from UNIPROT Identifiers
##### The labels come from the FASTA headers included in the FASTA database UP000002032 for Escherichia coli (strain B / BL21-DE3) merged with the possible contamination and exogenously expressed protein sequences. These FASTA files are included in the Initial filesdirectory.

##### Accession numbers are pulled from proteins in the database and contamination/ exogenously expressed proteins are flagged

In [3]:
def pull_acc(accession):
    if 'REV' in accession:
        return 'REV'
    elif 'CONT' in accession:
        return 'cont'
    elif 'EXTRA' in accession:
        return 'extra'
    elif 'REV' in accession:
        return 'REV'
    elif'|' in accession:
        return accession.split('|')[1]
    else:
        return 'unknown'
    
ori_df['Entry'] = ori_df['fasta_header'].apply(pull_acc)

# Remove the contamination and the incorrectly aligned entries
ori_df = ori_df[ori_df.Entry != 'cont']
ori_df = ori_df[ori_df.Entry != 'REV']

ori_df

,fasta_header,BL11,BL12,BL21,BL22,CF11,CF12,CF21,CF22,POOL1,...,POOL4,S111,S112,S121,S122,S711,S712,S721,S722,Entry
5,EXTRA_0117 (+1),1.440281e+07,1.976781e+07,1.777404e+07,1.323546e+07,1.061170e+07,1.441676e+07,1.598371e+07,1.290702e+07,3.680170e+07,...,3.689755e+07,1.271712e+07,1.192183e+07,1.637376e+07,1.248003e+07,1.228075e+08,1.244547e+08,1.007148e+08,1.100057e+08,extra
6,EXTRA_0118,4.354863e+08,4.575808e+08,4.548598e+08,4.165037e+08,3.489123e+08,4.174036e+08,3.975855e+08,3.707646e+08,8.356862e+08,...,8.453678e+08,1.743954e+09,1.499184e+09,1.547220e+09,1.508127e+09,1.096038e+09,1.057116e+09,1.002265e+09,1.096692e+09,extra
7,EXTRA_0119,1.539847e+08,2.001240e+08,1.980018e+08,1.596445e+08,1.080382e+08,2.495085e+08,1.974281e+08,1.836786e+08,8.359638e+08,...,8.412198e+08,2.061338e+09,1.934649e+09,1.918383e+09,1.835831e+09,1.232947e+09,1.280982e+09,1.098689e+09,1.286727e+09,extra
8,EXTRA_0121,1.500901e+07,1.762070e+07,1.691547e+07,1.469638e+07,1.207299e+07,1.455292e+07,1.614969e+07,1.252022e+07,3.134000e+07,...,3.136489e+07,1.166347e+07,1.177902e+07,1.510625e+07,1.237009e+07,8.914262e+07,9.935665e+07,8.252663e+07,8.880254e+07,extra
9,EXTRA_0125,3.332186e+07,3.613396e+07,3.704996e+07,3.125404e+07,5.372009e+07,5.407240e+07,5.730786e+07,5.168018e+07,6.714330e+07,...,6.536691e+07,3.139849e+07,3.339590e+07,3.510298e+07,3.100359e+07,1.433136e+08,1.434786e+08,1.489644e+08,1.757454e+08,extra
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1841,tr|A0A140SSC5|A0A140SSC5_ECOBD,2.947799e+06,1.997193e+06,2.225510e+06,2.449666e+06,2.298554e+06,2.014835e+06,1.812042e+06,1.978959e+06,1.999097e+06,...,1.950417e+06,1.750735e+06,1.652074e+06,1.641069e+06,1.746549e+06,1.701800e+06,1.677214e+06,1.916191e+06,1.604524e+06,A0A140SSC5
1842,tr|C6EGE8|C6EGE8_ECOBD,3.293522e+07,2.955776e+07,2.839105e+07,2.650044e+07,3.626475e+07,3.444847e+07,3.259868e+07,3.229483e+07,3.380914e+07,...,3.344520e+07,4.330823e+07,3.605778e+07,3.886553e+07,3.495989e+07,3.629801e+07,2.949572e+07,3.713997e+07,3.704146e+07,C6EGE8
1848,tr|A0A140N3T4|A0A140N3T4_ECOBD_family,9.913713e+07,1.185469e+08,1.178518e+08,8.869592e+07,8.172277e+07,9.162688e+07,1.089112e+08,7.855679e+07,2.432933e+08,...,2.408502e+08,7.315117e+07,6.734962e+07,9.698558e+07,7.824103e+07,7.371243e+08,7.150549e+08,7.386432e+08,8.135074e+08,A0A140N3T4
1849,EXTRA_0123_family,9.621529e+07,1.098033e+08,1.083949e+08,9.617536e+07,8.786624e+07,9.854661e+07,1.051216e+08,8.942986e+07,1.581107e+08,...,1.573009e+08,7.987325e+07,8.525064e+07,8.753340e+07,8.202509e+07,3.717637e+08,3.870576e+08,3.546169e+08,3.905663e+08,extra


### Map the names of the expressed proteins to their intensities

In [4]:
def expressed_protein_accession(row):
    entry = row['fasta_header']
    for string in exp_df['Entry']:
        if string in entry:
            return exp_accession[string]
    return row['Entry']

def find_expressed(row):
    entry = row['fasta_header']
    for string in exp_df['Entry']:
        if string in entry:
            print('found')
            return 'exp'
    return 'not_exp'

exp_df = pd.read_csv('../Initial files/expressed_protein_list.csv')
exp_entry = exp_df['Entry'].tolist()

exp_names = dict(zip(exp_df['Entry'],exp_df['Protein name']))
exp_accession = dict(zip(exp_df['Entry'],exp_df['Accession']))


ori_df['Entry'] = ori_df.apply(expressed_protein_accession, axis=1)

### Join the protein information with the normalized intensity data

In [5]:
ref_df = pd.read_csv('../Initial files/proteome_UP000002032_info.csv',index_col=False)

# Join the information from the proteome with the mass spectrometry data
combined_df = ori_df.set_index('Entry').join(ref_df.set_index('Entry'))


### Identify the proteins that were intentionally over-expressed

In [6]:
def expressed_protein_name(row):
    entry = row['fasta_header']
    for string in exp_df['Entry']:
        if string in entry:
            return exp_names[string]
    return row['Protein_names']

def find_expressed(name):
    if name in exp_prots:
        return 'exp'
    else:
        return 'not_exp'
    
exp_prots = exp_df['Protein name'].tolist()

combined_df['Protein_names'] = combined_df.apply(expressed_protein_name,axis=1)
combined_df['expressed'] = combined_df['Protein_names'].apply(find_expressed)



### Calculate comparisons between replicates and samples
##### These calculations will be used to generate the volcano plots in the R Jupyter Notebook titled "Extract comparison volcano plots"

In [7]:
def mean_calc(row, extracts=None):
    return np.mean([row[ex] for ex in extracts])

def std_calc(row,extracts=None):
    return np.std([row[ex] for ex in extracts])

def ttest_ind(row, extracts=None):
    '''
    The t-test used in this funnction is a two-tailed t-test with the 
    null hypothesis that the two sample means will be the same
    '''
    ex1_names = [extracts[0]+name for name in replicates]
    ex2_names = [extracts[1]+name for name in replicates]

    ex1 = [row[ex] for ex in ex1_names]
    ex2 = [row[ex] for ex in ex2_names]

    return stats.ttest_ind(ex1,ex2)[1]
    
extracts = ['BL','CF','S1','S7']
replicates = ['11','12','21','22']

for ex in extracts:
    ex_names = [ex+name for name in replicates]
       
    col_name = ex + '_mean'
    combined_df[col_name] = combined_df.apply(mean_calc, axis=1, extracts = ex_names)
    
    col_name = ex + '_std'
    combined_df[col_name] = combined_df.apply(std_calc, axis=1, extracts = ex_names)
    
pairs = [['CF','BL'],['S1','BL'],['S7','BL']]

for pair in pairs:
    diff_col = pair[0]+'_'+pair[1]+'_fc'
    combined_df[diff_col] = combined_df[pair[0]+'_mean']/combined_df[pair[1]+'_mean']
    ttest_col = pair[0]+'_'+pair[1]+'_p_val'
    combined_df[ttest_col] = combined_df.apply(ttest_ind, axis=1, extracts = pair)
combined_df

,fasta_header,BL11,BL12,BL21,BL22,CF11,CF12,CF21,CF22,POOL1,...,S1_mean,S1_std,S7_mean,S7_std,CF_BL_fc,CF_BL_p_val,S1_BL_fc,S1_BL_p_val,S7_BL_fc,S7_BL_p_val
Entry,,,,,,,,,,,,,,,,,,,,,
A0A140NC65,EXTRA_0117 (+1),1.440281e+07,1.976781e+07,1.777404e+07,1.323546e+07,1.061170e+07,1.441676e+07,1.598371e+07,1.290702e+07,3.680170e+07,...,1.337318e+07,1.756278e+06,1.144957e+08,9.725488e+06,0.827234,0.187037,0.820691,1.585444e-01,7.026417,2.750512e-06
A0A140N6W0,EXTRA_0118,4.354863e+08,4.575808e+08,4.548598e+08,4.165037e+08,3.489123e+08,4.174036e+08,3.975855e+08,3.707646e+08,8.356862e+08,...,1.574621e+09,9.941888e+07,1.063028e+09,3.856855e+07,0.869780,0.017993,3.569699,1.185702e-06,2.409905,2.307444e-07
A0A140NFD7,EXTRA_0119,1.539847e+08,2.001240e+08,1.980018e+08,1.596445e+08,1.080382e+08,2.495085e+08,1.974281e+08,1.836786e+08,8.359638e+08,...,1.937550e+09,8.069399e+07,1.224836e+09,7.576583e+07,1.037791,0.838927,10.888860,2.809213e-08,6.883470,4.375648e-07
A0A140N6C6,EXTRA_0121,1.500901e+07,1.762070e+07,1.691547e+07,1.469638e+07,1.207299e+07,1.455292e+07,1.614969e+07,1.252022e+07,3.134000e+07,...,1.272971e+07,1.398028e+06,8.995711e+07,6.032408e+06,0.860748,0.107949,0.792615,2.141029e-02,5.601179,8.075405e-07
A0A140N9R4,EXTRA_0125,3.332186e+07,3.613396e+07,3.704996e+07,3.125404e+07,5.372009e+07,5.407240e+07,5.730786e+07,5.168018e+07,6.714330e+07,...,3.272524e+07,1.645280e+06,1.528755e+08,1.339832e+07,1.573612,0.000030,0.950212,3.335119e-01,4.438899,5.338188e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A0A140SSC5,tr|A0A140SSC5|A0A140SSC5_ECOBD,2.947799e+06,1.997193e+06,2.225510e+06,2.449666e+06,2.298554e+06,2.014835e+06,1.812042e+06,1.978959e+06,1.999097e+06,...,1.697607e+06,5.120471e+04,1.724932e+06,1.160714e+05,0.842437,0.145869,0.705853,1.369417e-02,0.717215,1.908630e-02
C6EGE8,tr|C6EGE8|C6EGE8_ECOBD,3.293522e+07,2.955776e+07,2.839105e+07,2.650044e+07,3.626475e+07,3.444847e+07,3.259868e+07,3.229483e+07,3.380914e+07,...,3.829786e+07,3.224365e+06,3.499379e+07,3.190959e+06,1.155236,0.031770,1.305040,8.069681e-03,1.192450,4.836450e-02
A0A140N3T4,tr|A0A140N3T4|A0A140N3T4_ECOBD_family,9.913713e+07,1.185469e+08,1.178518e+08,8.869592e+07,8.172277e+07,9.162688e+07,1.089112e+08,7.855679e+07,2.432933e+08,...,7.893185e+07,1.111282e+07,7.510825e+08,3.723044e+07,0.850520,0.164570,0.744233,3.178442e-02,7.081812,1.260899e-07


In [8]:
combined_df.to_csv('../Intermediate data/labeled_IRS_proteins.csv')